# Getting the dataset

In [4]:
from datasets import load_dataset

ds = load_dataset("argilla/tripadvisor-hotel-reviews")

Generating train split:   0%|          | 0/20491 [00:00<?, ? examples/s]

In [11]:
sample_data = ds["train"]["text"][3]

In [12]:
print(sample_data)

great deal waikiki trip hawaii outrigger luana great hotel/condo, booked city view kitchenette, kitchen amenities ask, nice pool cabanas sun gets, barbeque area feel like getting fresh fish market highly recommended, staff friendly accommodating allowing access garage shower room 8 hours checked, 18/day parking little steep want explore life waikiki great hotel beginning main strip, tour desk not open probably went season desk touch tour desks outriggers waikiki happy make arrangements, beach minute walk away pay upwards 100/night right beach,  


# Building Prompt

In [18]:
import requests

API_URL = "http://nexusraven.nexusflow.ai"

headers = {
    "Content-Type": "application/json"
}

def query(payload):
    response = requests.post(API_URL, json=payload, headers=headers)
    return response.json()

def query_raven(prompt):
    payload = {
        "inputs": prompt,
        "parameters": {"do_sample": True, "temperature": 0.001, "max_new_tokens": 400, "stop": ["<bot_end>", "Thought:"], "return_full_text": False}
    }
    return query(payload)[0]["generated_text"].replace("Call:", "").replace("Thought:", "").strip()

# Defining what's important

In [14]:
from typing import List
from dataclasses import dataclass

In [15]:
schema_id = {"destination", "price", "hotel_facilities", "food", "sentiment", "nearby_places"}

In [16]:
dataclass_schema_representation = '''
@dataclass
class Record:
    destination: str # Destination of the hotel if provided, else ''
    price: str # Price of the hotel if provided, else ''
    hotel_facilities: List[str] # Facilities provided by the hotel if provided, else ''
    food: List[str] # Food provided by the hotel if provided, else ''
    sentiment: str # Overall customer sentiment, either 'frustrated' or 'happy'
    nearby_places: List[str] # Nearby locations to visit if provided, else ''
'''

In [17]:
exec(dataclass_schema_representation)

# Building the database

In [19]:
def initialize_db():
    import sqlite3

    # Connect to SQLite db
    conn = sqlite3.connect("tripadvisor.db")
    c = conn.cursor()

    # Table name
    table_name = "reviews"

    # Schema
    columns = '''
    id INTEGER PRIMARY KEY,
    destination TEXT,
    price TEXT,
    hotel_facilities TEXT,
    food TEXT,
    sentiment TEXT,
    nearby_places TEXT
    '''

    # Check if the table already exists
    c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    if c.fetchone() is None:
        # Create table
        c.execute(f"CREATE TABLE {table_name} ({columns})")

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

In [20]:
initialize_db()

# Tool to add data in the database

In [34]:
from dataclasses import dataclass, fields

def update_db(results: List[Record]):
    """
    Registers the information
    """
    import sqlite3
    from sqlite3 import ProgrammingError

    # Connect to SQLite db
    conn = sqlite3.connect("tripadvisor.db")
    c = conn.cursor()

    # Table name
    table_name = "reviews"

    # Insert records
    column_names = "destination, price, hotel_facilities, food, sentiment, nearby_places"
    placeholders = ", ".join(["?"] * 6)

    sql = f"INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})"

    for record in results:
        values = tuple(
            ",".join(getattr(record, field.name)) if isinstance(getattr(record, field.name), list)
            else getattr(record, field.name)
            for field in fields(record)
        )
        try:
            c.execute(sql, values)
        except ProgrammingError:
            print("Error with record.")
            pass

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

In [25]:
sample_record = Record(destination="hawaii", price="100/night", hotel_facilities=["kitchen", "barbeque", "pool", "garage", "rooms", "parking"], food=["fresh fish"], sentiment="happy", nearby_places=["beach"])

In [26]:
update_db([sample_record])

# Tool to extract from the database

In [35]:
import sqlite3

def execute_sql(sql: str):
    """
    Runs the SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible.
    """
    table_name = "reviews"

    # Connect to SQLite db
    conn = sqlite3.connect("tripadvisor.db")
    c = conn.cursor()

    # Execute the SQL query
    results = c.execute(sql).fetchall()

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

    return results

In [28]:
sql = '''SELECT * FROM reviews WHERE sentiment = "happy"'''

print("Executing SQL: ", sql)

execute_sql(sql)

Executing SQL:  SELECT * FROM reviews WHERE sentiment = "happy"


[(1,
  'hawaii',
  '100/night',
  'kitchen,barbeque,pool,garage,rooms,parking',
  'fresh fish',
  'happy',
  'beach')]

# Building the workflow

In [46]:
data = ds["train"]["text"][3]
data

'great deal waikiki trip hawaii outrigger luana great hotel/condo, booked city view kitchenette, kitchen amenities ask, nice pool cabanas sun gets, barbeque area feel like getting fresh fish market highly recommended, staff friendly accommodating allowing access garage shower room 8 hours checked, 18/day parking little steep want explore life waikiki great hotel beginning main strip, tour desk not open probably went season desk touch tour desks outriggers waikiki happy make arrangements, beach minute walk away pay upwards 100/night right beach,  '

In [47]:
import inspect

prompt = "\n" + data

sg = inspect.signature(update_db)
sg = str(sg).replace("__main__.Record", "Record")
docstring = update_db.__doc__

raven_prompt = f'''{dataclass_schema_representation}\nFunction:\n{update_db.__name__}{sg}:\n   """{docstring}"""\n\n\nUser Query:{prompt}<human_end>'''
print(raven_prompt)


@dataclass
class Record:
    destination: str # Destination of the hotel if provided, else ''
    price: str # Price of the hotel if provided, else ''
    hotel_facilities: List[str] # Facilities provided by the hotel if provided, else ''
    food: List[str] # Food provided by the hotel if provided, else ''
    sentiment: str # Overall customer sentiment, either 'frustrated' or 'happy'
    nearby_places: List[str] # Nearby locations to visit if provided, else ''

Function:
update_db(results: List[Record]):
   """
    Registers the information
    """


User Query:
great deal waikiki trip hawaii outrigger luana great hotel/condo, booked city view kitchenette, kitchen amenities ask, nice pool cabanas sun gets, barbeque area feel like getting fresh fish market highly recommended, staff friendly accommodating allowing access garage shower room 8 hours checked, 18/day parking little steep want explore life waikiki great hotel beginning main strip, tour desk not open probably went season de

In [48]:
raven_call = query_raven(raven_prompt)
print(raven_call)

update_db(results=[])


In [49]:
exec(raven_call)

In [70]:
prompt = "how many reviews are there where customers are happy"

sig = inspect.signature(execute_sql)

docstring = execute_sql.__doc__

sql_schema_representation = \
"""
CREATE TABLE reviews (
    id INTEGER PRIMARY KEY,
    destination TEXT,
    price TEXT,
    hotel_facilities TEXT,
    food TEXT,
    sentiment TEXT,
    nearby_places TEXT
)
"""

raven_prompt = f'''{sql_schema_representation}\nFunction:\n{execute_sql.__name__}{sig}:\n   """{docstring}"""\n\n\nUser Query:{prompt}<human_end>'''

print(raven_prompt)


CREATE TABLE reviews (
    id INTEGER PRIMARY KEY,
    destination TEXT,
    price TEXT,
    hotel_facilities TEXT,
    food TEXT,
    sentiment TEXT,
    nearby_places TEXT,
)

Function:
execute_sql(sql: str):
   """
    Runs the SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible.
    """


User Query:how many reviews are there where customers are happy<human_end>


In [71]:
raven_call = query_raven(raven_prompt)
print(raven_call)

execute_sql(sql='SELECT COUNT(*) FROM reviews WHERE sentiment = "happy"')


In [72]:
eval(raven_call)

[(6,)]

# Let's re-initialize and run the workflow

In [77]:
!rm tripadvisor.db
initialize_db()

In [78]:
from tqdm import tqdm

# Taking data points
data = ds["train"]["text"][:30]

for i in tqdm(range(0, 30)):

    # Ask raven to extract information we want out of the review
    raven_prompt = "\n" + data[i]
    sig = inspect.signature(update_db)
    docstring = update_db.__doc__
    raven_prompt = f'''{dataclass_schema_representation}\nFunction:\n{update_db.__name__}{sig}:\n   """{docstring}"""\n\n\nUser Query:{raven_prompt}<human_end>'''
    raven_call = query_raven(raven_prompt)
    exec(raven_call)

100%|██████████| 30/30 [00:31<00:00,  1.05s/it]


In [82]:
sig = inspect.signature(execute_sql)
docstring = execute_sql.__doc__
sql_schema_representation = \
"""
CREATE TABLE reviews (
    id INTEGER PRIMARY KEY,
    destination TEXT,
    price TEXT,
    hotel_facilities TEXT,
    food TEXT,
    sentiment TEXT,
    nearby_places TEXT
)
"""

raven_prompt = f'''{sql_schema_representation}\nFunction:\n{execute_sql.__name__}{sig}:\n   """{docstring}"""\n\n\nUser Query:{prompt}<human_end>'''
raven_prompt = raven_prompt + "User Query: How many frustrated customers?<human_end>"
print(raven_prompt)

raven_call = query_raven(raven_prompt)
print(raven_call)

eval(raven_call)


CREATE TABLE reviews (
    id INTEGER PRIMARY KEY,
    destination TEXT,
    price TEXT,
    hotel_facilities TEXT,
    food TEXT,
    sentiment TEXT,
    nearby_places TEXT
)

Function:
execute_sql(sql: str):
   """
    Runs the SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible.
    """


User Query:how many reviews are there where customers are happy<human_end>User Query: How many frustrated customers?<human_end>
execute_sql(sql='SELECT COUNT(*) FROM reviews WHERE sentiment = "frustrated";')


[(2,)]

In [85]:
raven_prompt = f'''{sql_schema_representation}\nFunction:\n{execute_sql.__name__}{sig}:\n   """{docstring}"""\n\n\nUser Query:{prompt}<human_end>'''
raven_prompt = raven_prompt + "User Query: Give me all the destination names and the facilites these hotels provide where customers are frustrated?<human_end>"
print(raven_prompt)

raven_call = query_raven(raven_prompt)
print(raven_call)

eval(raven_call)


CREATE TABLE reviews (
    id INTEGER PRIMARY KEY,
    destination TEXT,
    price TEXT,
    hotel_facilities TEXT,
    food TEXT,
    sentiment TEXT,
    nearby_places TEXT
)

Function:
execute_sql(sql: str):
   """
    Runs the SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible.
    """


User Query:how many reviews are there where customers are happy<human_end>User Query: Give me all the destination names and the facilites these hotels provide where customers are frustrated?<human_end>
execute_sql(sql='SELECT destination, hotel_facilities FROM reviews WHERE sentiment = "frustrated";')


[('airport hotel',
  'triparound,australia,arrived,place,rudest,receptionist,corridors,room,looked,like,bombsite,no,warning,room,mid,decoration'),
 ('', '')]